In [ ]:
import scipy.io
import os
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftshift
from scipy.signal import hilbert
from scipy.signal.windows import tukey 
import numpy as np
import vectorised_hf as vhf

# Load just the waveform from the data file
data = scipy.io.loadmat('data/ultrasound_reflection_data.mat', variable_names=['__header__', '__version__', '__globals__', 'elementPositions', 'samplingFrequency', 'soundSpeed','waveform', 'rcvData'])
waveform = data['waveform']
print(type(data))
print (data.keys())
folder_path = 'data'


In [ ]:
# Extract the numpy arrays from the dictionary using their variable names
waveform = data['waveform']
samplingFrequency = data['samplingFrequency']
elementPositions = data['elementPositions']
soundSpeed = data['soundSpeed']
rcvData = data['rcvData']


In [ ]:
Ntx, Nrx, Nt = rcvData.shape
rcvData2D = rcvData.reshape(Ntx*Nrx, Nt)
winData2D = vhf.tukey_vectorised(rcvData2D,  alpha=0.1, noise_Length=300)
test_env = vhf.envelope_detection(winData2D)

In [ ]:
plt.plot (test_env[1,:])

In [ ]:
# Test the create Imaging Grid function with dx = 0.0003 and Lx = 0.24
dx = 0.002 
Lx = 0.24
X, Y = vhf.createImagingVector(dx, Lx)
Npx = X.size
print(Npx)

In [ ]:
# Precompute the distance map here [m]
# Define parameters
dx = 0.001  # Grid spacing
Lx = 0.1    # Size of the grid
speedOfSound = 1540  # Speed of sound in m/s (typical for soft tissues)

# Create imaging vector
Xp, Yp = vhf.createImagingVector(dx, Lx)

# Detector coordinates (example values)
Xd = elementPositions[:,0]  # X coordinates of detectors
Yd = elementPositions[:,1]  # Y coordinates of detectors

# Xd, Xp, Yd and Yp are in 1D, this means that the function cannot run, since they don't have an overlapping dimension to be multipled by
# The following code makes the 'points' 2D and transposes the matrices so that they are in the right row to column order, before running the distance map function
Xd2D = Xd.reshape(-1,1)
Xp2D = Xp.reshape(-1,1)
Yd2D = Yd.reshape(-1,1)
Yp2D = Yp.reshape(-1,1)
Yd2DT = Yd2D.T
Xd2dT = Xd2D.T
print(Xp2D.shape)
print(Xd2dT.shape)
# Calculate distance map
distanceMap = vhf.calculateDistanceMap(Xd2dT, Yd2DT, Xp2D, Yp2D)
plt.imshow(distanceMap)
plt.set_aspect('equal','box')
# Convert distance map to time map
timeMap = vhf.timeMap(distanceMap, speedOfSound)

# Now timeMap contains the time taken for sound to travel 
# from each detector position to each pixel position

# Precompute the distance map here [m]

# Convert this to a delay map [s]

# Convert this to a sample index travel time map [samples]